# RNNとは

RNNが通常のニューラルネットワークと異なる点は内部状態を保持しているという点です。

ある時点$t$での入力$x(t)$と前の時点での内部状態$h(t - 1)$を入力すると新しい内部状態$h(t)$が得られ、この$h(t)$を更に線形層などで目的の出力$o(t)$に変換するという流れになります。

$x(1)$から$x(t)$までこれを繰り返していくと内部状態も$h(0)$から$h(1)$, $h(2)$, $h(3)$と次々に更新されていき、出力$o(t)$も現在の状態に依存して変化していきます。

![picture 1](img/RNN_5-1.png)

もっともシンプルなRNNであるElman Networkでは$x(t)$と$h(t-1)$から$h(t)$に変換する部分が線形層と活性化関数になっています。

上の図のようにRNNを時間方向に展開してみると、実はRNNはネットワークが一直線に積み重ねっており、FeedForward型のニューラルネットワークとおなじ構造をしていることに気づいたかもしれません。

しかしRNNは通常のニューラルネットワークよりも訓練することが一般的には困難です。

長い時間に渡る履歴を取り込もうと思うとそれだけ深いネットワークと同じになり勾配消失や勾配発散などの問題が発生します。

これを解決するものとして単純な線形層ではなく、より洗練されたいくつかの処理をまとめたモジュールのブロックで置き換えたLSTMやGRUなどのRNNも考案されています。

 ## テキストデータの数値化

 ここではテキストデータを扱う際によく使用する前処理や数値化の手法について紹介します。
 特に数値化では代表的なものとしてBoWとEmbeddingを扱います。

 RNNを実際に触る前にまずはテキストデータを数値に変換する方法について見ていきましょう。

 大きく分けて3つのステップに別れます。

 1. 正規化とトークン化
 2. 辞書の構築
 3. 数値への変換

 ### 正規化とトークン化

正規化とトークン化では文章を何らかの単位のリストに分割します。

例えば単語や文字などが分割単位として考えられます。

単語単位に分割する際には英語などの言語では単純にスペースで区切れば良いですが、日本語や中国語などの場合には形態素解析などの処理が必要になります。

トークン化と同時に表記ゆれなどの正規化も同時に行います。

### 辞書の構築

辞書の構築ではすべての文章の集合についてトークンを集め、数字のIDを降っていく作業です。またこのトークンを集めた集合のことをコーパスといいます。

### 数値への変換

最後の数値への変換はトークンのリストとして分割された文章を2で構築した辞書を使用してIDのリストに変換する作業です。

この一連の作業で1つの長い文字列だった文章が最終的には数値のリストに変換されます。

この数値のリストをさらに集計し。各IDの出現回数のベクトルとして表すものが Bag of Words（BoW）と呼ばれるものです。

例えば以下のようになります

(I, you, am, of, ...) = (1, 0, 1, 3)

BoWは計算がかんたんであり、複数の文章をまとめると粗行列として表現できるので非常に効率がよいというメリットがありますが、トークンの順番という重要な情報が失われてしまうという問題点もあります。

そこでニューラルネットワークではEmbeddingという手法でトークンをベクトルに変換し、ベクトルデータの時系列として文章を扱うことが主流です。

Pytorchでは nn.Embeddingを使用することでEmbedding層を作ることができます。

例えば全部で10000種類のトークンがあり、これを20次元ベクトルで表現する場合は以下のように記述します。

In [2]:
import torch
from torch import nn, optim

In [3]:
emb = nn.Embedding(10000, 20, padding_idx=0)

# Embedding層への入力はint64のTensor
inp = torch.tensor([1, 2, 5, 2, 10], dtype=torch.int64)

nn.Embeddingはpadding_idxを指定するk所とでそのIDはすべて0のベクトルに変換します。

辞書にないトークンはすべてIDを0にし、実際のIDは1からはじめるというように使用するとよいでしょう。

その場合のトークンの種類は0を含めた数をnn.Embeddingの1つ目引数に入力してください。 （なぜ）

なおnn.Embeddingも微分可能であり、これの内部の重みパラメータもネットワーク全体の訓練時に最適化することができますし、他の問題で事前に学習したnn.Embeddingを利用するということも可能です。